<p style="font-family: Arial; font-size:3.75em;color:purple; font-style:bold"><br>
Sudoku
</p><br>
<strong>The ultimate spoiler for a time-wasting game ;)</strong>
<p>Possible improvements:</p>
- JS front-end
- randomly create a new game
- identify the level of difficulty of a game

<img src="https://automatetheboringstuff.com/images/000037.jpg">

In [1]:
#We'll need an interface for user input (a csv template?)
#By default, the grid is converted to a table.
import pandas as pd
import numpy as np
dict_grid={
    'A':[5,6,'',8,4,7,'','',''],
    'B':[3,'',9,'','','',6,'',''],
    'C':['','',8,'','','','','',''],
    'D':['',1,'','',8,'','',4,''],
    'E':[7,9,'',6,'',2,'',1,8],
    'F':['',5,'','',3,'','',9,''],
    'G':['','','','','','',2,'',''],
    'H':['','',6,'','','',8,'',7],
    'I':['','','',3,1,6,'',5,9]
    
}
grid=pd.DataFrame(data=dict_grid)
grid

,A,B,C,D,E,F,G,H,I
0,5,3,,,7,,,,
1,6,,,1,9,5,,,
2,,9,8,,,,,6,
3,8,,,,6,,,,3
4,4,,,8,,3,,,1
5,7,,,,2,,,,6
6,,6,,,,,2,8,
7,,,,4,1,9,,,5
8,,,,,8,,,7,9


### Define variables for the 3x3 sub-grids
<p>Insert into a function: to be recalculated every time a new value is entered in the grid</p>

In [2]:
#example:
upper_left=grid.iloc[:3,:3]
#the subgrids will be numbered from 0 to 8, from top-left to bottom-right. upper_left corresponds to sub_grids[0]
#remove the empty cells, we need sub-grids only to generate a set of digits.
def content_sub_grids(grid):
    sub_grids=[]
    i=1
    while i <10:
        col=0
        while col<9:
            if i in [1,2,3]:
                row=0
            elif i in [4,5,6]:
                row=3
            elif i in [7,8,9]:
                row=6
            sub=grid.iloc[row:row+3,col:col+3]
            sub = set(filter(None, sub.values.flatten()))
            sub_grids.append(sub)
            col+=3
            i+=1
    return sub_grids
content_sub_grids(grid)[6]

{6}

### Define a class Cell with a value and sub-grid number

In [3]:
class Cell():
    def __init__(self, grid, x, y):
        '''grid is a pd.dataFrame'''
        '''x and y are integers within 0 and 8'''
        self.x=x
        self.y=y
        self.value = grid.iloc[x][y]
        self.id=str(x)+str(y)
    def find_sub_grid(self):
        if self.x<3:
            self.list_grids=[0,1,2]
        elif self.x<6:
            self.list_grids=[3,4,5]
        else:
            self.list_grids=[6,7,8]
        if self.y<3:
            return self.list_grids[0]
        elif self.y<6:
            return self.list_grids[1]
        else:
            return self.list_grids[2]
ex_cell=Cell(grid,5,8)
print(ex_cell.value)
print(ex_cell.find_sub_grid())

6
5


### Algorithm

In [4]:
def solve_for_cell(grid,cell,dict_possibles={}):
    forbidden=[]
    if cell.id in dict_possibles:
        possible=dict_possibles[cell.id]
    else:
        possible=list(range(1,10))
    #list digits in the row
    forbidden=set(filter(None,grid.loc[cell.x]))
    #list digits in the column
    forbidden.update(filter(None,set(list(grid[grid.columns[cell.y]]))))
    #list digits in the subgrid
    forbidden.update(content_sub_grids(grid)[cell.find_sub_grid()])
    #remove forbidden digits from a set of all possible digits (from 1 to 9)
    possible=set(possible).difference(forbidden)
    if len(possible)==1:
        grid.iloc[cell.x][cell.y]=list(possible)[0]
    else:
        dict_possibles[cell.id]=possible
solve_for_cell(grid,ex_cell)

In [5]:
def solve(grid):
    dict_possibles={}
    filled=len(list(filter(None,grid.values.flatten())))
    while filled<81:
        print(filled)
        for row in range(0,9):
            for col in range(0,9):
                if grid.iloc[row][col]=='':
                    cell=Cell(grid,row,col)
                    solve_for_cell(grid,cell,dict_possibles)
        filled=len(list(filter(None,grid.values.flatten())))  
    return grid

In [6]:
solve(grid)

30
36
48
66
78


,A,B,C,D,E,F,G,H,I
0,5,3,4,6,7,8,9,1,2
1,6,7,2,1,9,5,3,4,8
2,1,9,8,3,4,2,5,6,7
3,8,5,9,7,6,1,4,2,3
4,4,2,6,8,5,3,7,9,1
5,7,1,3,9,2,4,8,5,6
6,9,6,1,5,3,7,2,8,4
7,2,8,7,4,1,9,6,3,5
8,3,4,5,2,8,6,1,7,9
